<a href="https://colab.research.google.com/github/HIsu1231/AI_INOVATION_SQUARE/blob/master/CNN_5X5X32_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.4.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.0


In [3]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 23kB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 3.8MB 38.1MB/s 
     |████████████████████████████████| 512kB 42.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=1bdd8d662a17ff70b939c76e84a813dfb691212f13df462a7daf5fa709bc5e34
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

print("train images shape = ", mnist.train.images.shape)
print("train labels shape = ", mnist.train.labels.shape )
print("test images shape = ", mnist.test.images.shape)
print("test lables shape = ", mnist.test.labels.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [5]:
i_nodes = mnist.train.images.shape[1]
o_nodes = mnist.train.labels.shape[1]

#hyper_parameter
learning_rate = 0.001
epochs = 30
batch_size = 100

X = tf.placeholder(tf.float32, [None,i_nodes])
T = tf.placeholder(tf.float32, [None,o_nodes])

A1 = tf.reshape(X, [-1,28,28,1]) #28X28X1

In [6]:
#5X5X32 Filter
W2 = tf.Variable(tf.random_normal([5,5,1,32], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([32]))

#28X28X32
C2 = tf.nn.conv2d(A1, W2, strides =[1,1,1,1], padding='SAME')

Z2 = tf.nn.relu(C2+b2)

#14X14X32 (합성곱층에 zero-padding을 사용하며, 이 경우에는 출력 특성맵의 크기는 입력을 스트라이드로 나눈 다음 올림 한 것과 같다
A2 = tf.nn.max_pool(Z2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


A2_flat = tf.reshape(A2, [-1, 14*14*32])

In [7]:
W3 = tf.Variable(tf.random_normal([14*14*32,10],stddev=0.01))
b3 = tf.Variable(tf.random_normal([10]))

Z3 = tf.matmul(A2_flat,W3) + b3

A3 = tf.nn.softmax(Z3)

In [8]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [9]:
predicted_val = tf.argmax(A3,1)

predicted = tf.cast(tf.equal(tf.argmax(A3,1), tf.argmax(T,1)), dtype=tf.float32)

accuracy = tf.reduce_mean(predicted)



In [16]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    start_time = datetime.now()

    false_list = []

    for i in range(epochs):

        total_batch = int(mnist.train.num_examples / batch_size)

        for step in range(total_batch):

            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)

            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})  
            
            if step % 100 == 0:

                print("epochs = ", i, ", step = ", step, ", loss value = ", loss_val)

    end_time = datetime.now()

    print("\nElapsed time = ", end_time - start_time)


    test_x_data = mnist.test.images
    test_t_data = mnist.test.labels


    accuracy_val, predicted_list, index_label = sess.run([accuracy, predicted_val, predicted], feed_dict={X: test_x_data, T: test_t_data})

    index_label_list = list(index_label)
    print("Accuracy = ", accuracy_val)
    print("false label count = ",index_label_list.count([0]))

    temp_list = []
    index_label_prediction_list = []

    for i in range(len(index_label)):

      if index_label[i] == 0:

          temp_list.append(i)
          temp_list.append(np.argmax(test_t_data[i]))
          temp_list.append(predicted_list[i])

          index_label_prediction_list.append(temp_list)

          temp_list = []

epochs =  0 , step =  0 , loss value =  2.9748945
epochs =  0 , step =  100 , loss value =  0.7350495
epochs =  0 , step =  200 , loss value =  0.46770412
epochs =  0 , step =  300 , loss value =  0.3555438
epochs =  0 , step =  400 , loss value =  0.2542687
epochs =  0 , step =  500 , loss value =  0.19231388
epochs =  1 , step =  0 , loss value =  0.2186991
epochs =  1 , step =  100 , loss value =  0.0647089
epochs =  1 , step =  200 , loss value =  0.09715272
epochs =  1 , step =  300 , loss value =  0.17685626
epochs =  1 , step =  400 , loss value =  0.13786513
epochs =  1 , step =  500 , loss value =  0.17679475
epochs =  2 , step =  0 , loss value =  0.061761826
epochs =  2 , step =  100 , loss value =  0.16051227
epochs =  2 , step =  200 , loss value =  0.124439165
epochs =  2 , step =  300 , loss value =  0.04139178
epochs =  2 , step =  400 , loss value =  0.055228893
epochs =  2 , step =  500 , loss value =  0.16658299
epochs =  3 , step =  0 , loss value =  0.032243267
epo

In [17]:
print(index_label_prediction_list)

[[62, 9, 5], [232, 8, 5], [259, 6, 0], [340, 5, 3], [445, 6, 0], [449, 3, 5], [583, 2, 7], [684, 7, 3], [726, 7, 5], [870, 6, 5], [882, 9, 7], [883, 3, 5], [938, 3, 5], [947, 8, 9], [958, 3, 0], [1014, 6, 5], [1039, 7, 2], [1112, 4, 6], [1182, 6, 5], [1192, 9, 7], [1226, 7, 2], [1232, 9, 4], [1242, 4, 9], [1247, 9, 5], [1290, 3, 5], [1296, 6, 5], [1319, 8, 0], [1326, 7, 2], [1337, 2, 6], [1393, 5, 3], [1523, 8, 5], [1527, 1, 5], [1530, 8, 7], [1549, 4, 6], [1553, 9, 3], [1609, 2, 3], [1676, 6, 5], [1678, 2, 0], [1709, 9, 5], [1717, 8, 0], [1790, 2, 7], [1878, 8, 3], [1901, 9, 4], [1982, 6, 5], [2035, 5, 3], [2098, 2, 0], [2118, 6, 0], [2130, 4, 9], [2135, 6, 1], [2266, 1, 5], [2272, 8, 0], [2280, 3, 5], [2293, 9, 0], [2329, 0, 2], [2380, 9, 0], [2387, 9, 1], [2406, 9, 4], [2414, 9, 4], [2462, 2, 0], [2488, 2, 4], [2534, 3, 5], [2578, 7, 2], [2582, 9, 5], [2654, 6, 1], [2720, 9, 4], [2760, 9, 4], [2778, 4, 0], [2896, 8, 0], [2921, 3, 2], [2939, 9, 5], [2952, 3, 5], [2953, 3, 5], [2979, 